<a href="https://colab.research.google.com/github/frisk0zisan/nlp100/blob/main/4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 言語処理100本ノック　第4章

## 30. 形態素解析結果の読み込み  
形態素解析結果（neko.txt.mecab）を読み込むプログラムを実装せよ．ただし，各形態素は表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をキーとするマッピング型に格納し，1文を形態素（マッピング型）のリストとして表現せよ．第4章の残りの問題では，ここで作ったプログラムを活用せよ．

In [ ]:
!wget https://nlp100.github.io/data/neko.txt

--2021-01-05 04:23:28--  https://nlp100.github.io/data/neko.txt
Resolving nlp100.github.io (nlp100.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to nlp100.github.io (nlp100.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 975789 (953K) [text/plain]
Saving to: ‘neko.txt’

neko.txt            100%[===================>] 952.92K  --.-KB/s    in 0.06s   

2021-01-05 04:23:29 (15.7 MB/s) - ‘neko.txt’ saved [975789/975789]



In [ ]:
# 形態素分析ライブラリーMeCab と 辞書(mecab-ipadic-NEologd)のインストール 
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null

# シンボリックリンクによるエラー回避
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

fatal: destination path 'mecab-ipadic-neologd' already exists and is not an empty directory.
ln: failed to create symbolic link '/usr/local/etc/mecabrc': File exists


In [ ]:
!mecab -o ./neko.txt.mecab ./neko.txt

In [40]:
!head -10 ./neko.txt.mecab

一	名詞,数,*,*,*,*,一,イチ,イチ
	記号,一般,*,*,*,*,*
EOS
	記号,一般,*,*,*,*,*
EOS
　	記号,空白,*,*,*,*,　,　,　
吾輩	名詞,代名詞,一般,*,*,*,吾輩,ワガハイ,ワガハイ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
猫	名詞,一般,*,*,*,*,猫,ネコ,ネコ
で	助動詞,*,*,*,特殊・ダ,連用形,だ,デ,デ


In [ ]:
filename = "./neko.txt.mecab"
sentences = []
ans = []

with open(filename,"r") as f:
  for line in f:
    if line != "EOS\n": # 文末以外を追加
        fields = line.split('\t')
        if len(fields) != 2 or fields[0] == '':  # 文頭以外の空白と改行文字はスキップ
          continue
        else:
          word_dic = {"surface":fields[0], "base":fields[1].split(',')[6], "pos":fields[1].split(',')[0], "pos1":fields[1].split(',')[1] }
          ans.append(word_dic)
    else:
        sentences.append(ans)
        ans = []


In [41]:
for i in range(20):
  print(sentences[i])

[{'surface': '一', 'base': '一', 'pos': '名詞', 'pos1': '数'}]
[]
[{'surface': '\u3000', 'base': '\u3000', 'pos': '記号', 'pos1': '空白'}, {'surface': '吾輩', 'base': '吾輩', 'pos': '名詞', 'pos1': '代名詞'}, {'surface': 'は', 'base': 'は', 'pos': '助詞', 'pos1': '係助詞'}, {'surface': '猫', 'base': '猫', 'pos': '名詞', 'pos1': '一般'}, {'surface': 'で', 'base': 'だ', 'pos': '助動詞', 'pos1': '*'}, {'surface': 'ある', 'base': 'ある', 'pos': '助動詞', 'pos1': '*'}, {'surface': '。', 'base': '。', 'pos': '記号', 'pos1': '句点'}]
[{'surface': '名前', 'base': '名前', 'pos': '名詞', 'pos1': '一般'}, {'surface': 'は', 'base': 'は', 'pos': '助詞', 'pos1': '係助詞'}, {'surface': 'まだ', 'base': 'まだ', 'pos': '副詞', 'pos1': '助詞類接続'}, {'surface': '無い', 'base': '無い', 'pos': '形容詞', 'pos1': '自立'}, {'surface': '。', 'base': '。', 'pos': '記号', 'pos1': '句点'}]
[]
[{'surface': '\u3000', 'base': '\u3000', 'pos': '記号', 'pos1': '空白'}, {'surface': 'どこ', 'base': 'どこ', 'pos': '名詞', 'pos1': '代名詞'}, {'surface': 'で', 'base': 'で', 'pos': '助詞', 'pos1': '格助詞'}, {'surface': '生れ', 'bas

## 31. 動詞Permalink
動詞の表層形をすべて抽出せよ．

In [ ]:
verbs = []
for sentence in sentences:
  for word in sentence:
    if '動詞' == (word['pos']) :
      verbs.append(word["surface"])
print("重複有りの場合, 動詞の数 = {}".format(len(verbs)))

重複有りの場合, 動詞の数 = 28905


In [ ]:
verbs = list(set(verbs))
print("重複無しの場合, 動詞の数 = {}".format(len(verbs)))

重複無しの場合, 動詞の数 = 3893


## 32. 動詞の基本形Permalink
動詞の基本形をすべて抽出せよ．

In [ ]:
verbs_base = []
for sentence in sentences:
  for word in sentence:
    if '動詞' == (word['pos']) :
      verbs_base.append(word["base"])
print("重複有りの場合, 動詞の数 = {}".format(len(verbs_base)))

重複有りの場合, 動詞の数 = 28905


In [42]:
verbs_base = list(set(verbs_base))
print("重複無しの場合, 動詞の数 = {}".format(len(verbs_base)))

重複無しの場合, 動詞の数 = 2300
